# Find Eclipsing Binary Star Candidates from a Radial Velocity Catalog on Vizier: Solutions

***

In [1]:
%matplotlib inline
import numpy as np
from astropy.io import fits
from astropy.io import ascii
from astropy.table import Table
from astropy.coordinates import SkyCoord
from astropy import units as u
import matplotlib.pyplot as plt
from astroquery.mast import Observations
from astroquery.vizier import Vizier

***

## Exercises

1. An alternative method to narrowing down candidates from Table 4a: Instead of filtering by ```Per``` and sorting by qmin, filter by some qmin value (such as > 0.3) and then sort by period from lowest to highest. Still require Conv=1. How do your resulting tables vary? Which one do you think would be most helpful in trying to find eclipses, and why? 
2. An alternative method to searching astroquery in a loop: Export the APOGEE name, RA, and Dec from our filtered version of Table 4a (`tbl`) above to a .csv file, and use the Upload Target List feature on the online MAST Portal to search for QLP and TASOC observations in bulk. Hint: After cross-referencing your uploaded list to all MAST observations, filter (in the left hand column) by Mission: HLSP and Product Type: time series. Reference the [Writing Tables](https://docs.astropy.org/en/stable/io/ascii/write.html) page of the astropy documentation and [Search a List of Targets](https://outerspace.stsci.edu/display/MASTDOCS/Search+a+List+of+Targets) from MAST for help. Export the data (list of all HLSP light curves) to a local file and load it into this notebook. Compared to our loop above, which method is faster? Which is easier to find what you need?

In [2]:
# Exercise 1 Solution:
v2 = Vizier(columns=['APOGEE','Per','M1','M2min','qmin','RAJ2000', 'DEJ2000'],
           column_filters={"qmin":">0.3","Conv":"=1"})
v2.ROW_LIMIT = -1
catalogs2=v2.get_catalogs('J/AJ/156/18')
tbl2=catalogs2[2]
tbl2.sort('Per')
print(tbl2)
# Result: As of the writing of this tutorial, we have 24 stellar systems that meet our criteria, 
# but many of the periods are extremely long, so eclipses are unlikely to be observed by TESS.

      APOGEE          Per        M1     M2min     qmin    RAJ2000    DEJ2000  
                       d      solMass  solMass              deg        deg    
------------------ ---------- -------- -------- -------- ---------- ----------
2M00092789+0145417    2.61133  0.78317  0.40848  0.52157   2.366216   1.761607
2M00104203+0152065   23.47142  0.92806  0.36483  0.39311   2.675149   1.868474
2M19364967+3813244   24.72355  0.88201  0.31293  0.35479 294.206988  38.223454
2M20183197+1953430   51.30696  1.96604  1.36037  0.69193 304.633241  19.895283
2M01210284+8431304   54.25957  2.25258  0.96650  0.42906  20.261862  84.525131
2M04411627+5855354   56.05161  1.46516  0.61969  0.42295  70.317793  58.926521
2M07103169+0712585   62.37094  1.65363  0.58727  0.35514 107.632065   7.216252
2M07282763+2225408   80.29508  1.91156  0.97309  0.50906 112.115125  22.428013
2M00085727+7341257   83.66193  1.77608  1.12293  0.63225   2.238636  73.690475
2M20512287+5144445  108.33268  1.42243  1.07114  0.7

In [4]:
# Before the Exercise 2 solution, this portion of the code reproduces the variable "tbl" from the main notebook.
v=Vizier(columns=['APOGEE','Per','M1','M2min','qmin','RAJ2000', 'DEJ2000'],
           column_filters={"Per":"<100","Conv":"=1"})
v.ROW_LIMIT = -1
catalogs=v.get_catalogs('J/AJ/156/18')
tbl=catalogs[2]
tbl.sort('qmin')
tbl.reverse()
tbl=tbl[~tbl['qmin'].mask]
# Exercise 2 Solution:
ascii.write(tbl[['APOGEE','RAJ2000','DEJ2000']],'coords.csv',overwrite=True,delimiter=',')
# Outside of the Jupyter notebook, you'll need to upload that file to the MAST portal. 
# The default filename for the results is 'MAST_Crossmatch_All_Observati.csv'
# Be sure that file is in the same directory as this notebook. 
# At the time of the writing of this tutorial, the result table is 5 rows long. 
masttbl=ascii.read('MAST_Crossmatch_All_Observati.csv')
masttbl
# It seems the MAST portal is faster to actually perform the search, 
# but we do have to do more "by hand" to narrow down the observations to the ones we want.
# However, this now gives us the option to export a list of all the TESS light curves we might like, 
# which we can sort by object, 
# instead of having to query the archive individually each time. 

APOGEE,RAJ2000,DEJ2000,calib_level,obs_collection,obs_id,target_name,s_ra,s_dec,t_min,t_exptime,wavelength_region,filters,target_classification,t_obs_release,instrument_name,project,distance
str18,float64,float64,int64,str4,str53,int64,float64,float64,float64,int64,str7,str4,int64,float64,str10,str4,int64
2M00085727+7341257,2.238636,73.690475,4,HLSP,hlsp_qlp_tess_ffi_s0018-0000000373283207_tess_v01_llc,373283207,2.23859187586,73.6904753894,58790.18018644,1800,Optical,TESS,--,59185.12138895,Photometer,TESS,0
2M00085727+7341257,2.238636,73.690475,4,HLSP,hlsp_qlp_tess_ffi_s0025-0000000373283207_tess_v01_llc,373283207,2.23859187586,73.6904753894,58983.15378152,1800,Optical,TESS,--,59167.08523145,Photometer,TESS,0
2M08112409+3251415,122.850404,32.861553,4,HLSP,hlsp_qlp_tess_ffi_s0020-0000000165853060_tess_v01_llc,165853060,122.850386903,32.8615681619,58842.0158634,1800,Optical,TESS,--,59177.41143517,Photometer,TESS,0
2M00085727+7341257,2.238636,73.690475,4,HLSP,hlsp_qlp_tess_ffi_s0019-0000000373283207_tess_v01_llc,373283207,2.23859187586,73.6904753894,58815.59685655,1800,Optical,TESS,--,59181.05509258,Photometer,TESS,0
2M00085727+7341257,2.238636,73.690475,4,HLSP,hlsp_qlp_tess_ffi_s0024-0000000373283207_tess_v01_llc,373283207,2.23859187586,73.6904753894,58955.2999708,1800,Optical,TESS,--,59170.23827551,Photometer,TESS,0


***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 